# Day 23 
## Part 1
The longest path problem is NP complete unless it's a DAG, so I'll assume that's the case where the slopes enforce a lack of loops.

In [4]:
from dataclasses import dataclass

@dataclass
class Point:
    x: int
    y: int

    def __add__(self, other):
        return self.__class__(self.x + other.x, self.y + other.y)

    def __sub__(self, other):
        return self.__class__(self.x - other.x, self.y - other.y)

    def __neg__(self):
        return self.__class__(-self.x, -self.y)

    def __hash__(self):
        return hash((self.x, self.y))

    def __lt__(self, other):
        if self.x < other.x:
            return True
        elif self.x > other.x:
            return False
        else:
            return self.y < other.y

    def __iter__(self):
        yield self.x
        yield self.y

    def __mod__(self, other):
        if isinstance(other, Point):
            return self.__class__(self.x % other.x, self.y % other.y)
        else:
            return self.__class__(self.x % other, self.y % other)
        
    def __mul__(self, multiple):
        return self.__class__(self.x * multiple, self.y * multiple)
    

N = Point(0, 1)
S = Point(0, -1)
W = Point(-1, 0)
E = Point(1, 0)

DIRECTIONS = (N, E, S, W)

In [68]:
slopes = {
    ">": E,
    "<": W,
    "v": S,
    "^": N
}

import networkx as nx
from collections import deque

def parse_data(s):
    grid = {}
    lines = s.strip().splitlines()
    for y, line in zip(range(len(lines) - 1, -1, -1), lines):
        for x, c in enumerate(line):
            if c != "#":
                grid[Point(x, y)] = c
    return grid

def create_dag(grid):
    # create a DAG
    max_y = max(p.y for p in grid)
    start = next(p for p in grid if p.y == max_y)
    end = next(p for p in grid if p.y == 0)

    G = nx.DiGraph()
    G.add_edge(start, start + S)
    q = deque([(start, start + S)])

    while q:
        a, b = q.popleft()
        for d in DIRECTIONS:
            n = b + d
            if (
                n in grid and
                n != a and
                (grid[b] not in slopes or slopes[grid[b]] == d) 
                and (grid[n] not in slopes or slopes[grid[n]] == d)
            ):
                G.add_edge(b, n)
                q.append((b, n))

    return G

def part_1(data):
    return nx.dag_longest_path_length(create_dag(data))
    
test_data = parse_data("""#.#####################
#.......#########...###
#######.#########.#.###
###.....#.>.>.###.#.###
###v#####.#v#.###.#.###
###.>...#.#.#.....#...#
###v###.#.#.#########.#
###...#.#.#.......#...#
#####.#.#.#######.#.###
#.....#.#.#.......#...#
#.#####.#.#.#########v#
#.#...#...#...###...>.#
#.#.#v#######v###.###v#
#...#.>.#...>.>.#.###.#
#####v#.#.###v#.#.###.#
#.....#...#...#.#.#...#
#.#########.###.#.#.###
#...###...#...#...#.###
###.###.#.###v#####v###
#...#...#.#.>.>.#.>.###
#.###.###.#.###.#.#v###
#.....###...###...#...#
#####################.#""")

assert part_1(test_data) == 94

In [70]:
data = parse_data(open("input").read())

part_1(data)

2294

## Part 2

This is no longer a DAG, so this is NP. However there are a lot of corridors where there's no choice for direction. Create a graph which connects only nodes with more than two neighbours.

In [133]:
def create_graph(grid):
    # create a graph
    max_y = max(p.y for p in grid)
    start = next(p for p in grid if p.y == max_y)
    end = next(p for p in grid if p.y == 0)

    G = nx.Graph()
    G.add_edge(start, start + S)
    q = deque([(start, start + S)])

    for p in grid:
        for d in DIRECTIONS:
            n = p + d
            if n in grid:
                G.add_edge(p, n, weight=1)

    # compress the graph to connect only nodes not in corridors
    for p in grid:
        nbrs = list(G[p])
        if len(nbrs) == 2:
            a, b = nbrs
            G.add_edge(a, b, weight = G[p][a]["weight"] + G[p][b]["weight"])
            G.remove_node(p)

    # compress further to remove dead ends
    for p in grid:
        if p in G and p != start and p != end:
            nbrs = list(G[p])
            if len(nbrs) == 1:
                G.remove_node(p)

    return start, end, G

In [134]:
start, end, g = create_graph(test_data)

In [135]:
g.nodes

NodeView((Point(x=1, y=22), Point(x=11, y=19), Point(x=3, y=17), Point(x=21, y=11), Point(x=5, y=9), Point(x=13, y=9), Point(x=13, y=3), Point(x=19, y=3), Point(x=21, y=0)))

In [123]:
[nx.path_weight(g, path, "weight") for path in nx.all_simple_paths(g, start, end)]

[150, 110, 82, 126, 94, 86, 118, 82, 74, 154, 118, 90]

In [142]:
def part_2(data):
    start, end, g = create_graph(data)
    max_so_far = 0
    for m in (
        nx.path_weight(g, path, "weight") 
        for path in nx.all_simple_paths(g, start, end)
    ):
        if m > max_so_far:
            print(m)
            max_so_far = m
    return max_so_far

In [143]:
assert part_2(test_data) == 154

150
154


In [145]:
%%time 

part_2(data)

4662
4726
5034
5102
5166
5254
5362
5394
5582
5726
5794
5850
5894
5910
5986
5998
6142
6214
6246
6342
6418


KeyboardInterrupt: 

I've completely cheated here, I've left this running and just punted the max so far into the answer and it worked. The search needs pruning. If I'm feeling eager, which I doubt, I'll implement that. Even using networkx that took from Newcastle to Hitchin to solve.